# UCSC Toil Workflow Execution Service Demonstration

<img src="https://github.com/dockstore/dockstore-ui2/raw/develop/src/assets/images/sponsors/coloured/ga4gh.png" width="180" align="right"/>

<img src="https://cgl.genomics.ucsc.edu/wp-content/uploads/2017/07/TOIL-Slug-Logo-862x1116.jpg" width="200" align="right"/>

This notebook is meant to contain all of the necessary parts needed to install and run the [Workflow Execution Service](https://github.com/ga4gh/workflow-execution-service-schemas) backed by [UCSC Toil](https://toil.readthedocs.io/en/3.15.0/) on a linux compliant system with Python installed!

The Workflow Execution Service attempts to present the interface for workflow execution over HTTP methods. Simple JSON requests including the inputs and outputs for a workflow are sent to a service. This allows us to "ship code to the data," since data privacy and egress costs require that data is not shared.

UCSC Toil is software for executing workflows. It presents a Python native API, which will not be demonstrated here, as well as a CWL compliant CLI interface. For that reason, any CWLRunner can easily be exposed by the workflow-service, demonstrated here.

## Installing the Dependencies

[Docker](https://docs.docker.com/install/) is a required dependency to make workflow execution portable in this example. So install that first.

Once you have docker installed, you can follow the below instructions, which will use Python's package manager to download the requirements.

In [ ]:
!pip install toil git+git://github.com/common-workflow-language/workflow-service

If you have trouble executing that line, try putting it into a terminal. Depending on your Python installation, you may need to enter your password.

## Starting the Server

Now that you have toil and the workflow-service installed, you just have to turn on the server, and it will be ready to accept requests!

We'll have to tell the service which runner to use, the CWL runner which comes with the service, and the optional tool to use to run it, in this case `cwltoil`. Lastly, we lower the log output of toil so that the output JSON can be read by the wes-server (and returned to the client).

In [5]:
!wes-server --backend=wes_service.cwl_runner --opt runner=cwltoil --opt extra=--logLevel=CRITICAL

 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
^C


Background processes aren't supported directly in notebooks, so we close it here. But you can paste this command in a terminal and it will bring up your very own Workflow Execution Service!

## Using the Client

The server is now running, but Toil hasn't started yet as we haven't issued any Workflow Execution requests. Here, using the provided CLI client, we demonstrate a simple workflow which calculates an md5sum.

The workflow description is provided in the workflow-service test data, and we specify local inputs and outputs. `File` is currently the only supported file system protocol of the workflow-service.

### Accessing a workflow via Dockstore Tool Registry Service

We will start by accessing the metadata for a workflow from dockstore.

#### List Tools


In [17]:
import requests
response = requests.get('https://dockstore.org:8443/api/ga4gh/v1/tools/', params={"name": "md5sum"})
print(response.json()[0]['versions'][0])
md5sum_url = response.json()[0]['versions'][0]['url'] + '/plain-CWL/descriptor/%2FDockstore.cwl'

{u'verified': True, u'name': u'master', u'url': u'https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fbriandoconnor%2Fdockstore-tool-md5sum/versions/master', u'image': u'7f82fc51fa35d36bbd61297ee0c05170ab4ba67c969a9a66b28e5ed3c100034b', u'meta-version': u'2017-07-23 15:45:37.0', u'descriptor-type': [u'CWL', u'WDL'], u'dockerfile': True, u'id': u'quay.io/briandoconnor/dockstore-tool-md5sum:master', u'verified-source': u'Phase 1 GA4GH Tool Execution Challenge'}


We now have a URL we can pass too WES for execution!

In [18]:
print(md5sum_url)

https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fbriandoconnor%2Fdockstore-tool-md5sum/versions/master/plain-CWL/descriptor/%2FDockstore.cwl


### Using the WES CLI client to Execute

In [19]:
!wes-client --host localhost:8080 --proto http $md5sum_url testdata/md5sum.cwl.json

INFO:root:Workflow id is 1b7cbeac80e84740a450f2c6bc12b7f2
INFO:root:State is COMPLETE
INFO:root:
{
    "output_file": {
        "format": "http://edamontology.org/data_3671", 
        "checksum": "sha1$5cd16de143136d95a0307bc1db27d88b57b033e9", 
        "basename": "md5sum.txt", 
        "nameext": ".txt", 
        "nameroot": "md5sum", 
        "http://commonwl.org/cwltool#generation": 0, 
        "location": "file:///home/david/git/workflow-service/workflows/1b7cbeac80e84740a450f2c6bc12b7f2/outdir/md5sum.txt", 
        "class": "File", 
        "size": 33
    }
}

As you can see, the wes-client routed a request and polled the service until the its state was `COMPLETE`. It then shows us the location of the outputs, so we can read them.

In [20]:
!cat /home/david/git/workflow-service/workflows/1b7cbeac80e84740a450f2c6bc12b7f2/outdir/md5sum.txt

b1946ac92492d2347c6235b4d2611184


## Future Work

Because toil implements the cwl CLI interface, it can be easily exchanged for a number of CWL runners. Although this demonstration works only for local files, it should be possible to demonstrate provisioners like those in Toil.

Both the workflow-service and Toil are Python native applications, and this suggests a deeper integration is possible. Future demonstrations like these could use native Python code to interact with WES.

Dockstore and the Tool Registry Service API could be used to first find the workflow that will be run, demonstrating interoperability in these services.

By provisioning using DOS URLs, it should be possible for systems to reason about file locations whether they are system local or on a cloud.